In [15]:
import sys
import os
import pandas as pd
import warnings
import json
from pydantic import BaseModel

# Add the src folder to the path
sys.path.append(os.path.abspath('..'))
warnings.filterwarnings("ignore", category=FutureWarning)

# Import the classes
from src.data_tool.yahoo_finance import yahoo_finance
from src.data_tool.get_data_fd import get_prices, get_financial_metrics, search_line_items, get_insider_trades, get_company_news
from src.data_tool.ploygon import polygon
from src.data_tool.financial_datasets import FinancialDatasets
from src.data_tool.connect_db import ConnectDB
from src.data_tool.data_models import polygon_news_response

from src.llm.api_call import call_llm
from langchain_core.prompts import ChatPromptTemplate

db =ConnectDB()
engine = db.get_engine()

DB_USER and DB_PASSWORD are set


In [16]:
fd = FinancialDatasets()
pl = polygon()

DB_USER and DB_PASSWORD are set
POLYGON_API_KEY is set


In [17]:
ticker = 'NVDA'
start_date = '2025-03-01'
end_date = '2025-04-02'

In [18]:
def prepare_news_for_llm(news_list):
    """
    Prepare company news data for an LLM by removing URLs and converting to a structured format.
    
    Args:
        news_list: List of CompanyNews objects
        
    Returns:
        List of dictionaries containing cleaned news data
    """
    processed_news = []
    
    for news_item in news_list:
        # Create a dictionary with all fields except URL
        news_dict = {
            "tickers": news_item.tickers,
            "title": news_item.title,
            "description": news_item.description,
            "published_utc": news_item.published_utc,
        }
        
        processed_news.append(news_dict)
    
    return processed_news

In [19]:
nvdia_news, nvdia_news_df = pl.get_news(ticker, start_date, end_date, limit=1000, strict=False)

In [20]:
poly_news = polygon_news_response(**nvdia_news)

In [21]:
LLM_news = prepare_news_for_llm(poly_news.results)

In [22]:
template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.
                """,
            ),
            (
                "human",
                """Based on the following news, create the investment signal:

                News Data for {ticker}:
                {llm_news}

                Return the trading sentiment in the following JSON format exactly:
                {{
                  "sentiment": "bullish" | "bearish" | "neutral",
                  "confidence": float between 0 and 100,
                  "reasoning": "string"
                }}
                """,
            ),
        ]
    )

prompt = template.invoke({"llm_news": json.dumps(LLM_news, indent=2), "ticker": ticker})

In [23]:
print(prompt)

messages=[SystemMessage(content='You are a financial analyst. You are given a list of news articles and a ticker. You need to analyze the news and provide a summary of the news and the sentiment of the news.\n                ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Based on the following news, create the investment signal:\n\n                News Data for NVDA:\n                [\n  {\n    "tickers": [\n      "NVDA"\n    ],\n    "title": "Why XRP Is Slipping Today",\n    "description": "XRP\'s token price is down 2.4% as the cryptocurrency market experiences a pullback. Investors are concerned about factors like tariffs, inflation, and valuation profiles for risk-heavy investments, leading to a sell-off in XRP and other cryptocurrencies.",\n    "published_utc": "2025-03-01T00:05:27Z"\n  },\n  {\n    "tickers": [\n      "NVDA",\n      "IBIT"\n    ],\n    "title": "Billionaires Sell Nvidia Stock and Buy a BlackRock ETF Wall Street Experts Say May Soar Up to 1

In [24]:
class sentiment(BaseModel):
    sentiment: str
    confidence: float
    reasoning: str


In [25]:
model_name = "gemini-2.0-flash"
model_provider = "Gemini"
pydantic_model = sentiment
max_retries = 3

In [26]:
sentiment_result = call_llm(prompt, model_name, model_provider, pydantic_model, max_retries=max_retries)
print(sentiment_result)

sentiment='neutral' confidence=65.0 reasoning="The news surrounding NVDA is mixed. There's positive news regarding partnerships, potential for growth in areas like automotive and AI data centers, and analysts projecting potential upside. However, there are also concerns about increasing competition, declining gross margins, potential regulatory challenges in China, and some major investors reducing their positions. The recent stock sell-off and lackluster IPO of Nvidia-backed CoreWeave add to the uncertainty. Overall, the bullish and bearish signals are relatively balanced, leading to a neutral sentiment."


In [27]:
sentiment_result

sentiment(sentiment='neutral', confidence=65.0, reasoning="The news surrounding NVDA is mixed. There's positive news regarding partnerships, potential for growth in areas like automotive and AI data centers, and analysts projecting potential upside. However, there are also concerns about increasing competition, declining gross margins, potential regulatory challenges in China, and some major investors reducing their positions. The recent stock sell-off and lackluster IPO of Nvidia-backed CoreWeave add to the uncertainty. Overall, the bullish and bearish signals are relatively balanced, leading to a neutral sentiment.")

In [28]:
print(type(sentiment_result))

<class '__main__.sentiment'>
